In [1]:
# Required libraries: BeautifulSoup and Selenium for webscraping,
# Pandas for collecting and exporting data
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

In [2]:
# Establish web driver, open webpage, and create dataframe
options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(10)
driver.get('https://utmbmontblanc.com/en/page/349/results.html')

df = pd.DataFrame(columns=['Place', 'Name', 'Gender', 'Time'])

# If there is a pop-up, close out of it
try:
    driver.find_element(
        'xpath',
        '//button[@aria-label="Close"]'
    ).click()
except:
    pass

# Navigate to correct set of results
driver.find_element(
    'xpath',
    '//a[@name="ct107"]'
).click()

driver.find_element(
    'xpath',
    '''
    //a[@href="javascript:see('edPass','2021','utmb')"]
    '''
).click()

driver.find_element(
    'xpath',
    '//table[@id="tabPass"]'
)

# Parse results with BeatifulSoup for ease
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
driver.quit()

table = soup.find('tbody')

# Iterate through rows in the results table, saving info to dataframe
for row in table.find_all('tr'):
    col = row.find_all('td')
    col = [td.text for td in col]
    if col[-1] != '\xa0':
        place = int(col[0])
        name = col[2].strip('\n')
        category = col[3]
        if category[-1]=='F':
            gender = 'F'
        elif category[-1]=='H':
            gender = 'M'
        else:
            gender = 'X'
        time = col[-1]
        df.loc[len(df.index)] = [place, name, gender, time]

# Export results and close webdriver
df.set_index('Place', drop=True, inplace=True)
df.to_csv('2021utmb.csv')
driver.quit()